In [1]:
import os
from tensorflow import keras
from keras import layers
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from DataHandling.features import slices
from DataHandling.features.slices import read_tfrecords,feature_description
from DataHandling import utility
from DataHandling.models import predict

2022-12-04 07:56:56.419672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 07:56:56.760059: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-04 07:56:57.664872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64${LD_LIBRARY_PATH:+:/usr/lib/cuda/include:/usr/lib/cuda/lib64:}
2022-12-04 07:56:57.664947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [2]:
os.environ['WANDB_DISABLE_CODE']='True'
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[-1], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

2022-12-04 07:57:00.133427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 07:57:00.160546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 07:57:00.160892: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [12]:
var=['u_vel',"v_vel","w_vel","pr0.025"]
target=['pr0.025_flux']
normalized=False
y_plus=30

In [13]:

file_path = "/home/yuning/thesis/valid/scratch"
path_test = os.path.join(file_path,"test")
print(path_test)

/home/yuning/thesis/valid/scratch/test


In [14]:
dataset = tf.data.TFRecordDataset(
                                    filenames=path_test,
                                    compression_type="GZIP",
                                    num_parallel_reads=tf.data.experimental.AUTOTUNE
                                    )


In [15]:
feature_dict = feature_description(file_path)
feature_dict

{'u_vel': FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
 'v_vel': FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
 'w_vel': FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
 'pr0.025': FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
 'pr0.025_flux': FixedLenFeature(shape=[], dtype=tf.string, default_value='')}

In [16]:
dataset = dataset.map(lambda x: read_tfrecords(x,feature_dict,target))

In [17]:
dataset.snapshot

<bound method DatasetV2.snapshot of <MapDataset element_spec=({'pr0.025': TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), 'u_vel': TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), 'v_vel': TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), 'w_vel': TensorSpec(shape=<unknown>, dtype=tf.float64, name=None)}, TensorSpec(shape=<unknown>, dtype=tf.float64, name=None))>>

In [10]:
u_vel = []
v_vel = []
w_vel = []
pr0025 = []
pr0025_flux = []
for ele in dataset.as_numpy_iterator():
    print(ele[0].keys())
    
    # tuple(dict) thus ele[0] we get dict and accroding to its key we git value
    # u_vel.append(ele[0]["u_vel"])
    # v_vel.append(ele[0]["v_vel"])
    # w_vel.append(ele[0]["w_vel"])
    # pr0025.append(ele[0]["pr0.025"])
    # pr0025_flux.append(ele[0]["pr0.025_flux"])
    break

dict_keys(['pr0.025', 'u_vel', 'v_vel', 'w_vel'])


In [39]:
print("There are {}  snapshots of u in test dataset ".format(len(u_vel)))
print("There are {}  snapshots of v in test dataset ".format(len(v_vel)))
print("There are {}  snapshots of w in test dataset ".format(len(w_vel)))
print("There are {}  snapshots of pr0.025 in test dataset ".format(len(pr0025)))

There are 407  snapshots of u in test dataset 
There are 407  snapshots of v in test dataset 
There are 407  snapshots of w in test dataset 
There are 407  snapshots of pr0.025 in test dataset 


In [40]:
import numpy as np 
U_data = np.array(u_vel)
V_data = np.array(v_vel)
W_data = np.array(w_vel)
Pr_data = np.array(pr0025)
print("There are {}  snapshots of u in test dataset ".format(U_data.shape))
print("There are {}  snapshots of v in test dataset ".format(V_data.shape))
print("There are {}  snapshots of w in test dataset ".format(W_data.shape))
print("There are {}  snapshots of pr0.025 in test dataset ".format(Pr_data.shape))

There are (407, 256, 256)  snapshots of u in test dataset 
There are (407, 256, 256)  snapshots of v in test dataset 
There are (407, 256, 256)  snapshots of w in test dataset 
There are (407, 256, 256)  snapshots of pr0.025 in test dataset 


In [95]:
U_TF = tf.data.Dataset.from_tensor_slices(U_data)
U_TF = U_TF.batch(batch_size=8)
U_TF = U_TF.shuffle(buffer_size=100)
U_TF = U_TF.repeat(2)
U_TF = U_TF.prefetch(3)

U_TF.snapshot

<bound method DatasetV2.snapshot of <PrefetchDataset element_spec=TensorSpec(shape=(None, 256, 256), dtype=tf.float64, name=None)>>

In [97]:
# tf.data.experimental.save(U_TF,os.path.join(file_path,"u_vel"),compression="GZIP")

In [98]:
# def custom_shard_func(element):
#   return np.int64(0)
tf.data.experimental.save(U_TF,os.path.join(file_path,"u_vel"))

In [99]:
DATA_U = tf.data.Dataset.load(os.path.join(file_path,"u_vel"))

In [100]:
print(U_TF.snapshot)
print(DATA_U.snapshot)

<bound method DatasetV2.snapshot of <PrefetchDataset element_spec=TensorSpec(shape=(None, 256, 256), dtype=tf.float64, name=None)>>
<bound method DatasetV2.snapshot of <_LoadDataset element_spec=TensorSpec(shape=(None, 256, 256), dtype=tf.float64, name=None)>>


In [101]:
for ele in DATA_U.as_numpy_iterator():
    print(ele.shape)
    break

(8, 256, 256)
